In [18]:
import sqlite3
from sqlite3 import Error
import datetime as datetime
import pandas as pd
import os
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import string
#from enum import Enum
#import statistics

In [19]:
"""
create a database connection to the SQLite3 database specified by the db_file
"""
db_file = "chat_20200515_1313.db"
conn = None
try:
    conn = sqlite3.connect(db_file)
except Error as e:
    print(e)

In [53]:
"""
Query message data and load into a dataframe for analysis
"""

#for person: where chat_identifier = '+12038099727' \
chat_query = "select text, id, date, display_name, is_from_me \
            from ( \
                select * from message \
                join ( \
                    select * \
                    from chat_message_join \
                    JOIN chat on chat_message_join.chat_id = chat.ROWID \
                    ) as t \
                ON message.ROWID = t.message_id \
                where display_name = 'Clown Test' \
                ) as t2 \
            left join handle on t2.handle_id = handle.ROWID \
            order by date \
            ;"

df = pd.read_sql_query(chat_query, conn)

#convert to US/Eastern time
imessage_epoch = datetime.datetime.strptime('01.01.2001 00:00:00,00', '%d.%m.%Y %H:%M:%S,%f')
df['date'] = [datetime.datetime.strptime(datetime.datetime.fromtimestamp((t / 1000000.0 + imessage_epoch.timestamp() * 1000) / 1000.0).strftime('%Y-%m-%d %H:%M:%S.%f'), '%Y-%m-%d %H:%M:%S.%f') + datetime.timedelta(hours=-5) for t in df['date']]
#df['date'] = [d.tz_localize('US/Eastern') for d in df['date']]

#pd.set_option("display.max_rows", 200, "display.max_columns", 20, "display.max_colwidth", 22)
#df.tail(1000)

In [54]:
lens = []
num_words = []
num_unique_words = []
reacts = []

member_to_word_usage = { i : {} for i in df.id.unique() } #TODO: change 'None' person to me
'''
word | person1 | person2 | ...
----   -------   -------   ...
hello     2         0
world     4        12
....
'''

resp_times = []
last_time = datetime.datetime.now() #placeholder

for index, row in df.iterrows():
    text = str(row['text'])
    
    #type of reaction (if any)
    msg_parts = text.split("“")
    if msg_parts[0] == 'Liked ':
        reacts.append('Like')
    elif msg_parts[0] == 'Disiked ':
        reacts.append('Disike')
    elif msg_parts[0] == 'Emphasized ':
        reacts.append('Emphasize')
    elif msg_parts[0] == 'Questioned ':
        reacts.append('Question')
    elif msg_parts[0] == 'Loved “':
        reacts.append('Love')
    elif msg_parts[0] == 'Laughed at ':
        reacts.append('Laugh')
    else:
        reacts.append('N/A')
    
    if reacts[index] == 'N/A':
        #length of each raw msg
        lens.append(len(text))

        '''CLEAN TEXT'''
        # split into words
        tokens = text.split()
        # convert to lower case
        tokens = [w.lower() for w in tokens]
        # remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        words = [word for word in stripped if word.isalpha()]
        # filter out stop words 
        #stop_words = set(stopwords.words('english')) #TODO: add more words to ignore like 'ur'
        #words = [w for w in words if not w in stop_words]

        #number words
        num_words.append(len(words))

        unique = set()
        for w in words:
            #number of unique words
            unique.add(w)

            #number of occurrances for words in this message
            if member_to_word_usage[row['id']].get(w) == None:
                member_to_word_usage[row['id']][w] = 1
            else:
                member_to_word_usage[row['id']][w] = member_to_word_usage[row['id']][w] + 1

        num_unique_words.append(len(unique))
   
    else: #reaction message
        lens.append(1)
        num_words.append(1)
        num_unique_words.append(1)
        #number of occurrances for words in this message
        if member_to_word_usage[row['id']].get(words[0]) == None:
            member_to_word_usage[row['id']][words[0]] = 1
        else:
            member_to_word_usage[row['id']][words[0]] = member_to_word_usage[row['id']][words[0]] + 1
        
    #time between messages
    if not index == 0:
        resp_times.append(row['date'] - last_time)
    if index == len(df.index) - 1:
        resp_times.append(datetime.timedelta(days=0, hours=0, minutes=0, seconds=0, microseconds=0)) #placeholder
    last_time = row['date']
             
df['length'] = lens
df['num_words'] = num_words
df['num_unique_words'] = num_unique_words
df['reaction'] = reacts
df['response_time'] = resp_times

df.tail(1000)

,text,id,date,display_name,is_from_me,length,num_words,num_unique_words,reaction,response_time
0,Test,+17816407410,2019-05-22 12:40:34.483829,Clown Test,0,4,1,1,N/A,00:00:27.035051
1,LMAO,+16178946439,2019-05-22 12:41:01.518880,Clown Test,0,4,1,1,N/A,00:00:00.143559
2,did you mean to send test here or clown toothp...,+17818126643,2019-05-22 12:41:01.662439,Clown Test,0,51,10,10,N/A,00:00:04.380629
3,i’m geeking,+16178946439,2019-05-22 12:41:06.043068,Clown Test,0,11,1,1,N/A,00:00:17.709894
4,I can’t say Greg,+17816407410,2019-05-22 12:41:23.752962,Clown Test,0,16,3,3,N/A,00:00:15.102433
...,...,...,...,...,...,...,...,...,...,...
208,Liked “Test”,+16178946439,2020-03-24 14:15:12.512317,Clown Test,0,1,1,1,Like,01:13:08.126797
209,Liked “Test”,+17816407410,2020-03-24 15:28:20.639114,Clown Test,0,1,1,1,Like,01:41:50.365740
210,Liked “Test”,+17816409567,2020-03-24 17:10:11.004854,Clown Test,0,1,1,1,Like,00:08:42.804337
211,Liked “Test”,+17819603282,2020-03-24 17:18:53.809191,Clown Test,0,1,1,1,Like,00:23:06.312839


In [41]:
#pd.set_option("display.max_rows", 200, "display.max_columns", 20, "display.max_colwidth", 22)
#print(df)


'''STATS'''
print(df['response_time'].sum() / (len(df.index) - 2)) #avg time between msgs

for x in member_to_word_usage:
    sorted_dict = {k: v for k, v in sorted(member_to_word_usage[x].items(), key=lambda item: item[1], reverse=True)}
    print (x)
    print (list(sorted_dict.items())[0: 100])
    print('-------------------------------------------------------------------------------------------')



1 days 10:56:35.666531
+17816407410
[('disliked', 10), ('test', 9), ('none', 3), ('i', 2), ('say', 2), ('the', 2), ('an', 2), ('greg', 1), ('know', 1), ('if', 1), ('you', 1), ('get', 1), ('beta', 1), ('all', 1), ('can', 1), ('is', 1), ('keep', 1), ('eye', 1), ('on', 1), ('your', 1), ('faulty', 1), ('loved', 1), ('someone', 1), ('text', 1), ('me', 1), ('personally', 1), ('that', 1), ('message', 1), ('hit', 1), ('for', 1), ('image', 1)]
-------------------------------------------------------------------------------------------
+16178946439
[('test', 18), ('disliked', 9), ('the', 4), ('for', 3), ('that', 3), ('faulty', 2), ('try', 2), ('again', 2), ('a', 2), ('of', 2), ('message', 2), ('hit', 2), ('lmao', 1), ('geeking', 1), ('greg', 1), ('saif', 1), ('you', 1), ('passed', 1), ('chig', 1), ('but', 1), ('like', 1), ('d', 1), ('loved', 1), ('someone', 1), ('text', 1), ('me', 1), ('personally', 1), ('gonna', 1), ('run', 1), ('out', 1), ('licenses', 1), ('already', 1), ('near', 1), ('end', 1)